In [1]:
from bd import *
from rs import *
from  scrapper import *
import requests

 ········


In [2]:
existeEnListTittleList("ls052277902")

False

**Obtener películas de una lista**

In [5]:
def get_movies(idList):      
    sql  = "SELECT id_tittlelist FROM list_tittlelist where id_list=%s "
    with conn.cursor() as cursor:
        cursor.execute( sql, (idList) )
        value = cursor.fetchall()
        if len(value) <= 0:
            return None

    movies = []
    for m in value: movies.append(m[0])
    return movies

get_movies("ms2020_ls1001")

['Da 5 Bloods',
 'Nomadland',
 'Lovers Rock',
 'The Trial of the Chicago 7',
 'First Cow',
 'Mank',
 'Never Rarely Sometimes Always',
 'Minari',
 "Ma Rainey's Black Bottom"]

**Obtener listas que contienen una película**

In [4]:
def get_lists(idMovie):      
    sql  = "SELECT id_list FROM list_tittlelist where id_tittlelist=%s "
    with conn.cursor() as cursor:
        cursor.execute( sql, (idMovie) )
        value = cursor.fetchall()
        if len(value) <= 0:
            return None

        lists = []
        for l in value: lists.append(l[0])
    return lists

get_lists("Grass")

['ms2019_ls1025', 'ms2019_ls1051', 'ms2019_ls1092', 'ms2019_ls1104']

**Función para cargar listas asociadas a una película**

In [5]:
import re
def load_lists(idTitle):

    lists = obtenerListasDesdeTitulo(idTitle)
    for l in lists:
        x = re.search(r"([0-9]+)", l[2])
        size = int(x.group())
        if size < 100 and size > 1:
            x = re.search(r"(ls\w+)", l[1])
            list_name = x.group()
            print(x.group(), len)
            if not existeEnListTittleList(list_name):
                descargarGuardarInfoList(list_name)



Esta función retorna las listas asociadas a una película.
Si la película tiene menos de 10 listas asociadas se intentan descargar.

In [6]:
def get_lists_or_download(idMovie, min_lists=10):
    lists = get_lists(idMovie)
    if len(get_lists(idMovie)) > min_lists: return lists
    
    load_lists(idMovie)
    return get_lists(idMovie)
      
#lists = get_lists_or_download('tt0403692')

Random Walk
--

Iteración de un random walk (lista aleatoria -> película aleatoria -> lista aleatoria)

Experimentos con metascore
---


In [2]:
with conn.cursor() as cursor:
        # ELIMINA VALORES EN TABLA
        sql = "DELETE FROM list_tittlelist"
        cursor.execute( sql )
        conn.commit()

Agrega las listas desde el archivo

In [4]:
import csv

with open("metascore2020.txt") as tsv:
    for mlist in csv.reader(tsv, dialect="excel-tab"):
        while '' in mlist: mlist.remove('')
        id_list = mlist[0]
        if existeEnListTittleList(id_list): continue
        for movie in mlist[1:-1]:
            #print(id_list, movie[0:29])
            instertarListTittleList( id_list, movie[0:29])

In [9]:
import random 
import numpy as np

def random_walk(movie, t=0.2, block_list=None):
    r=0.0
    movies = set()
    while r<=t:
        L = get_lists(movie)
        if block_list in L: L.remove(block_list)
        if len(L)==0: return movies
        r1 = random.randint(0, len(L)-1) 
        #se obtienen las películas de una lista aleatoria
        M = get_movies(L[r1])
        #se obtienen las listas de una película aleatoria
        r2 = random.randint(0, len(M)-1)
        movie=M[r2]
        movies.add(movie)
        r = random.uniform(0.0,1.0)
    return movies        

In [11]:
def recommend(source_movies, block_list=None, max_iter=1000, max_listsize=5, t=0.2):
    counters = dict()
    recommended_movies = set()

    for i in range(max_iter):
        for k in range(len(source_movies)):
            movies = random_walk(source_movies[k], block_list=block_list, t=t)
                
            for m in movies:
                if m not in counters:
                    counters[m] = np.zeros(len(source_movies), dtype=int)
                    counters[m][k] = 1
                else:
                    if np.min(counters[m])==5 and m not in source_movies and m not in recommended_movies: 
                        recommended_movies.add(m)
                        if len(recommended_movies) == max_listsize:
                            return recommended_movies, counters
                    counters[m][k] += 1
    
    #criterio para seleccionar
    
    
    return recommended_movies, counters
rec, counters  = recommend(["Monos","Ema"])
print (rec)

{'Glory', 'Marriage Story', 'Parasite', 'The Irishman', 'The Souvenir'}


In [12]:
#counters

In [13]:
all_movies=[]
id=1001
while existeEnListTittleList("ms2019_ls"+str(id)):
    all_movies += get_movies("ms2019_ls"+str(id))
    id += 1

In [14]:
def validate(list_name, nsample=3, npred=5, t=0.2):
    rw_tot_success = 0
    rand_tot_success = 0
    tot_size = 0
    
    M = get_movies(list_name) #se obtienen las pelis de la lista
    

    for m in M: 
        if m in all_movies: all_movies.remove(m)
            
    if verbose: print("movies:", M)

    #se filtran las películas que solo aprecen en la lista M
    M = [x for x in M if x in all_movies]
    
    if len(M)>=nsample+2:
    
        for i in range(1):
            # recommendation using random walk
            source_movies = random.sample(M,nsample) #seleccionamos nsample pelis al azar
            if verbose: print("source_movies:",source_movies)

            num = np.minimum(npred,len(M)-nsample) #número de películas a recomendar
            recommendations,_ = recommend(source_movies, max_listsize=npred, block_list=list_name, t=t) 
            rw_tot_success += len(recommendations.intersection(M))/float(num)
            tot_size += len(recommendations)
            if verbose: print("rw_recommendations:",recommendations)

            # random recommendation

            recommendations = random.sample(all_movies,npred)
            recommendations = set(recommendations)
            MM = [x for x in M if x not in source_movies]
            rand_tot_success += len(recommendations.intersection(MM))/float(num)
            if verbose: print("r_recommendations",recommendations)
        
        print(list_name +"\t"+ str(rand_tot_success/1.0) +"\t"+ str(rw_tot_success/1.0) +"\t" \
              + str(tot_size/1.0) + "\t" + str(len(M)) )

    M = get_movies(list_name) 
    for m in M: all_movies.append(m)

#verbose=False
#print("lista \tr_success \trw_success \t#recom \t lenM")
#for i in range(1,246):
#    validate("ms2019_ls"+str(1000+i), t=0.1, nsample=5, npred=10)

In [19]:
verbose=True
validate("ms2019_ls1001", t=0.1, nsample=5, npred=10)

movies: ['A Hidden Life', 'Honey', 'Honey Boy', 'John Wick: Chapter 3 - Parabe', 'Knives Out', 'Parasite', 'The Irishman', 'The Lighthouse', 'The Report', 'The Reports on Sarah and Sale', 'Too Late to Die Young', 'Uncut Gems']
source_movies: ['The Irishman', 'Parasite', 'Knives Out', 'Honey Boy', 'A Hidden Life']
rw_recommendations: {'The Farewell', 'Once Upon a Time in Hollywood', 'Booksmart', 'Marriage Story', 'Glory', 'Little Women', 'Uncut Gems', 'Portrait of a Lady on Fire', 'The Last Black Man in San Fra', 'Ad Astra'}
r_recommendations {'The Farewell', 'Doctor Sleep', 'The Irishman', 'Glory', 'The Beach Bum', 'Hampton', 'Apollo 11', 'The Last Black Man in San Fra', 'Joker'}
ms2019_ls1001	0.0	0.2	10.0	10


In [18]:
rec, counters  = recommend(['The Irishman', 'Parasite', 'Knives Out', 'Honey Boy', 'A Hidden Life'], max_listsize=10)

In [19]:
rec

{'Booksmart',
 'Glory',
 'Joker',
 'Little Women',
 'Marriage Story',
 'Once Upon a Time in Hollywood',
 'Portrait of a Lady on Fire',
 'The Farewell',
 'The Last Black Man in San Fra',
 'Uncut Gems'}

In [20]:
counters['A Hidden Life']

array([ 4,  2,  3,  9, 46])

In [26]:
for x in ['John Wick: Chapter 3 - Parabe', 'The Lighthouse', 'The Report', 'The Reports on Sarah and Sale', 'Too Late to Die Young']:
    print (counters[x])

[6 3 6 7 3]
[ 5 16 12 16 16]
[ 4  0  0  2 11]
[0 0 1 3 0]
[0 0 1 6 2]


In [27]:
for x in rec:
    print (counters[x])

[11 28 13 15 19]
[11 10 13 10  7]
[20 13  5  6 26]
[31 29 35 40 22]
[20 17 20 24  8]
[ 6 13  9 19  6]
[44 44 41 32 33]
[13  8 10 20  6]
[18 23 15 22 13]
[ 7 16 14 12 13]
